# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [231]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [232]:
pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [233]:
import glob
import io
import base64
import random
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [234]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/root/miniconda3/envs/env/lib/python3.12/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">
- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [235]:
# для картинки A

states = set(range(3))
actions = set(range(1))

print(f'{states=} | {actions=}')

T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}

A_mdp = states, actions, T

states_b = set(range(4))
actions_b = set(range(3))

T_b = {
    (0, 0) : 1,
    (0, 1) : 2,
    (0, 2) : 3,
    
    (1, 0): 1, (1, 1): 1, (1, 2): 2,
    (2, 0): 2, (2, 1): 2, (2, 2): 2,
    (3, 0): 3, (3, 1): 3, (3, 2): 3
}

B_mdp = states_b, actions_b, T_b


states_c = set(range(4))
actions_c = set(range(2))

T_c = {
    (0, 0): 1,
    (0, 1): 2,
    
    (1, 0): 1, (1, 1): 3,
    (2, 0): 3, (2, 1): 2,
    (3, 0): 3, (3, 1): 3
}

C_mdp = states_c, actions_c, T_c

states={0, 1, 2} | actions={0}


Попробуйте записать функцию перехода в матричном виде:

In [236]:
P = np.array([
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 0.0, 1.0]
])

P_b = np.array([
    [0.0, 1/3, 1/3, 1/3],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])

P_c = np.array([
    [0.0, 1/2, 1/2, 0.0],
    [0.0, 1/2, 0.0, 1/2],
    [0.0, 0.0, 1/2, 1/2],
    [0.0, 0.0, 0.0, 1.0]
])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [237]:
# возвести матрицу в n-ю степень
def state_after_n_steps(s, n, prob_m):
    for _ in range(n):
        s = np.matmul(s, prob_m)
        
    return s

state_a = state_after_n_steps([1, 0, 0], 100, P)
state_b = state_after_n_steps([1, 0, 0, 0], 100, P_b)
state_c = state_after_n_steps([1, 0, 0, 0], 1000, P_c)

print(state_a, state_b, state_c, sep='\n')


[0. 0. 1.]
[0.         0.33333333 0.33333333 0.33333333]
[0.00000000e+000 9.33263619e-302 9.33263619e-302 1.00000000e+000]


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [238]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))
    


В качестве дополнения, запишите стратегию агента

In [239]:
class OptimAgentB:
    def __init__(self, actions):
        self.actions = np.array(list(actions))
        
    def act(self, state):
        if state == 0:
            return random.randint(0, 2)
        return state

class OptimAgentC:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        if state == 0:
            return random.randint(0, 1)
        if state == 1:
            return 1
        if state == 2:
            return 0
        return random.randint(0, 1)

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [240]:
R = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}

R_B = {
    (0, 0): 1.0,
    (0, 1): 1.0,
    (0, 2): 1.0,
    (1, 0): 0.0, 
    (1, 1): 0.0,
    (1, 2): 0.0,
    (2, 0): 0.0, 
    (2, 1): 0.0,
    (2, 2): 0.0,
    (3, 0): 0.0, 
    (3, 1): 0.0,
    (3, 2): 0.0,
}
R_C = {
    (0, 0): 1.0,
    (0, 1): 1.0,
    (1, 0): 0.0,
    (1, 1): 1.0,
    (2, 0): 1.0,
    (2, 1): 0.0,
    (3, 0): 0.0,
    (3, 1): 0.0,
}
A_mdp = *A_mdp, R
B_mdp = *B_mdp, R_B
C_mdp = *C_mdp, R_C

## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [241]:
def run_episode(mdp, wAgent, termState):
    states, actions, T, R = mdp
    agent = wAgent(actions)
    s = 0
    tau = []
    for _ in range(5):
        if s == termState:
            break
        
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp, Agent, 2)

[(0, np.int64(0), -0.1), (1, np.int64(0), 1.0)]

In [242]:
run_episode(B_mdp, OptimAgentB, 3)


[(0, 2, 1.0)]

In [243]:
run_episode(C_mdp, OptimAgentC, 3)

[(0, 0, 1.0), (1, 1, 1.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [251]:
N = 100

def compute_G(tau, gamma=0.9):
    G = 0.0
    for t, (s, a, r) in enumerate(tau):
        G += (gamma ** t) * r
    return G

def mean_return(mdp, agent, term_state, n_runs=N, gamma=0.95):
    return np.mean([compute_G(run_episode(mdp, agent, term_state), gamma) for _ in range(n_runs)])

G_a = mean_return(A_mdp, Agent, 2)
G_b = mean_return(B_mdp, OptimAgentB, 3)
G_c = mean_return(C_mdp, OptimAgentC, 3)

print(G_a, G_b, G_c, sep='\n')

0.8499999999999995
1.0
1.949999999999999
